<a href="https://colab.research.google.com/github/mzamarron/DeepLearning/blob/main/Preparing_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library import

In [1]:
import os
import cv2
import h5py
import glob 
import zipfile
import shutil
import numpy as np
from tqdm import tqdm
from pathlib import Path
from sklearn.utils import shuffle

# Data import

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

file_path = '/content/drive/MyDrive/Colab_Notebooks/DeepLearning/Datasets/'

Mounted at /content/drive/


## Unzip dataset

In [3]:
local_zip = '/content/drive/MyDrive/Colab_Notebooks/DeepLearning/Datasets/Concrete.zip'

zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('')
zip_ref.close()

# Label creation

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
categories = ['Positive', 'Negative']
X = []
Y = []

for category in tqdm(categories):    
  imgs_path = glob.glob(file_path + 'concrete_cracks/'+ category +'/*.jpg')
  for ruta_img in imgs_path:
    img = cv2.resize(cv2.imread(ruta_img), (150, 150))
    X.append(img)
    if category == 'Negative':
      Y.append(0)
    else:
      Y.append(1)

100%|██████████| 2/2 [00:07<00:00,  3.80s/it]


In [6]:
# Changing data into an np array
X = np.asarray(X).astype('float32')
Y = np.expand_dims(np.asarray(Y).astype('float32'), axis = 1)

In [7]:
print('Images dimension: ', X.shape)
print('Labels: ', Y.shape)

Images dimension:  (200, 150, 150, 3)
Labels:  (200, 1)


# Order shuffling

In [8]:
# Shuffling data to mix order of positive and negative cracks
X, Y = shuffle(X,Y)

# Dataset creation

In [9]:
hdf5_dir = Path(file_path)

In [10]:
def store_many_hdf5(images, labels):
    """ Stores an array of images to HDF5.
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    
    # Create a new HDF5 file
    with h5py.File(hdf5_dir / f"concrete_dataset.h5", 'w') as hf:
      dataset = hf.create_dataset('images', data=images, shape=np.shape(images), compression='gzip', chunks=True)
      meta_set = hf.create_dataset('meta', data=labels, shape=np.shape(labels), compression='gzip', chunks=True)

In [11]:
store_many_hdf5(X, Y)